In [1]:
!pip install dbnomics
!pip install xlrd
!pip install openpyxl
import pandas as pd
import dbnomics
import requests
import http.client
import json
import time
import datetime
from datetime import datetime ## There is a class "datetime" in "datetime module", so this line is to allow us to do datetime.strptime() instead of datetime.datetime.strptime()
from dbnomics import fetch_series, fetch_series_by_api_link

  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=beb15e2c63f14234d9c5d84a33cdf1942455ef9f611b3d3c89c363ecf3c7cdb4
  Stored in directory: c:\users\promvarp\appdata\local\pip\cache\wheels\56\ea\58\ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt


In [7]:
# Import data from BOT API

## set end period (default id the latest available)
end_period = time.strftime('%Y-%m-%d')
########end_period = '2021-07-12'

## List of series, code, and start period of the importing items
quarterly_list  = {'Series': [  'ca_milUSD',        'ca_milTHB',   'port_in_milUSD',   'port_in_milTHB',  'port_out_milUSD',  'port_out_milTHB',  'port_net_milUSD', 'port_net_milTHB',        'bop_milUSD',       'bop_milTHB',         'SM',        'NPL',      'house_p',  'townhouse_p',       'land_p',      'condo_p'], 
        'code': [        'XTBOP0CA00Q02152', 'XTBOP0CA00Q01904', 'XTBOP0FA00Q02284', 'XTBOP0FA00Q02036', 'XTBOP0FA00Q02257', 'XTBOP0FA00Q02009', 'XTBOP0FA00Q02256','XTBOP0FA00Q02008',  'XTBOP0OVL0Q02371', 'XTBOP0OVL0Q02123', 'FINPQ00228', 'FINPQ00229', 'EIRPPIQ00081', 'EIRPPIQ00083', 'EIRPPIQ00087', 'EIRPPIQ00085'],
        'start_period': [      '2005-01-01',       '2005-01-01',       '2005-01-01',       '2005-01-01',       '2005-01-01',       '2005-01-01',       '2005-01-01',      '2005-01-01',        '2005-01-01',       '2005-01-01', '2007-01-01', '2007-01-01',   '2008-01-01',   '2008-01-01',   '2008-01-01',   '2008-01-01']}  
quarterly_list = pd.DataFrame(quarterly_list)  

monthly_list  = {'Series': ['res_gross_milUSD', 'res_gross_milTHB',          'gov2',          'gov5',         'gov10',           'NLR'], 
        'code': [           'XTRSV00000M00698', 'XTRSV00000M00704', 'FMRTINTM00031', 'FMRTINTM00034', 'FMRTINTM00039', 'FMRTINTM00172'],
        'start_period': [         '1993-01-01',      '1993-01-01',     '2005-01-01',    '2005-01-01',    '2005-01-01',    '2005-01-01']}  
monthly_list = pd.DataFrame(monthly_list)
monthly_list_93 = monthly_list.loc[0:1]
monthly_list_05 = monthly_list.loc[2:]


## function to load BOT api data
def load_api_data(name, code, start, end):
    conn = http.client.HTTPSConnection("apigw1.bot.or.th", 5050,timeout=300)

    headers = {
        'x-ibm-client-id': "0262b443-cd34-41f3-8a06-81f7725eacc8",
        'accept': "application/json"
        }
    
    request_text = "/bot/public/observations/?series_code=" + code +"&start_period=" + start + "&end_period=" + end + "&sort_by=asc"
    conn.request("GET", request_text , headers=headers)
    res = conn.getresponse()
    data = res.read()
    jsdata = json.loads(data)
    temp_data = pd.DataFrame(jsdata['result']['series'][0]['observations']).rename(columns={'value': name})
    return(temp_data)


## Quarterly data
quarterly_data  = load_api_data(quarterly_list['Series'][0], quarterly_list['code'][0], quarterly_list['start_period'][0], end_period)
for index, row in quarterly_list.iterrows():
    if quarterly_list['Series'][0] == row['Series']: continue
    temp_data = load_api_data(name = row['Series'], code = row['code'], start = row['start_period'], end = end_period)
    quarterly_data = pd.merge(quarterly_data, temp_data, on="period_start",how='left')
quarterly_data = quarterly_data.set_index('period_start')
    
    
## Monthly data
### BOT API limits query to 120 rows each time so we need to divide query into smaller batch (10y each)
for index, row in monthly_list_93.iterrows():
    data_93 = load_api_data(name = row['Series'], code = row['code'], start = '1993-01-01', end = '1993-02-01')
    data_93 = data_93.set_index('period_start')
    start_period_monthly = '1993-02-01'
    end_period_monthly = '2003-01-01'
    temp_data = load_api_data(name = row['Series'], code = row['code'], start = start_period_monthly, end = end_period_monthly)
    temp_data = temp_data.set_index('period_start')
    data_93 = data_93.append(temp_data)

    start_period_monthly = '2003-01-01'
    end_period_monthly = '2013-01-01'
    temp_data = load_api_data(name = row['Series'], code = row['code'], start = start_period_monthly, end = end_period_monthly)
    temp_data = temp_data.set_index('period_start')
    data_93 = data_93.append(temp_data)

    start_period_monthly = '2013-01-01'
    end_period_monthly = '2021-01-01'
    temp_data = load_api_data(name = row['Series'], code = row['code'], start = start_period_monthly, end = end_period_monthly)
    temp_data = temp_data.set_index('period_start')
    data_93 = data_93.append(temp_data)

    start_period_monthly = '2021-01-01'
    temp_data = load_api_data(name = row['Series'], code = row['code'], start = start_period_monthly, end = end_period)
    temp_data = temp_data.set_index('period_start')
    data_93 = data_93.append(temp_data)
    if 'data_93_main' in locals():
        data_93_main = pd.merge(data_93_main,data_93, on="period_start",how='left')
    else:
        data_93_main = data_93

for index, row in monthly_list_05.iterrows():
    data_05 = load_api_data(name = row['Series'], code = row['code'], start = '2005-01-01', end = '2005-02-01')
    data_05 = data_05.set_index('period_start')
    start_period_monthly = '2005-02-01'
    end_period_monthly = '2015-01-01'
    temp_data = load_api_data(name = row['Series'], code = row['code'], start = start_period_monthly, end = end_period_monthly)
    temp_data = temp_data.set_index('period_start')
    data_05 = data_05.append(temp_data)

    start_period_monthly = '2015-01-01'
    end_period_monthly = '2021-01-01'
    temp_data = load_api_data(name = row['Series'], code = row['code'], start = start_period_monthly, end = end_period_monthly)
    temp_data = temp_data.set_index('period_start')
    data_05 = data_05.append(temp_data)

    start_period_monthly = '2021-01-01'
    temp_data = load_api_data(name = row['Series'], code = row['code'], start = start_period_monthly, end = end_period)
    temp_data = temp_data.set_index('period_start')
    data_05 = data_05.append(temp_data)
    if 'data_05_main' in locals():
        data_05_main = pd.merge(data_05_main,data_05, on="period_start",how='left')
    else:
        data_05_main = data_05
monthly_data = pd.merge(data_93_main,data_05_main, on="period_start",how='left')


## Seriescode
### current account mil usd : series_code=XTBOP0CA00Q02152&start_period=2005-01-01&end_period=2021-01-01&sort_by=asc
### current account mil thb : series_code=XTBOP0CA00Q01904&start_period=2005-01-01&end_period=2021-01-01&sort_by=asc
### portfolio inflow (liab) USD: series_code=XTBOP0FA00Q02284&start_period=2005-01-01&end_period=2021-01-01&sort_by=asc", headers=headers)
### portfolio inflow (liab) THB: series_code=XTBOP0FA00Q02036&start_period=2005-01-01&end_period=2021-01-01&sort_by=asc", headers=headers)
### portfolio outflow (asset) USD: series_code=XTBOP0FA00Q02257&start_period=2005-01-01&end_period=2021-01-01&sort_by=asc", headers=headers)
### portfolio outflow (asset) THB: series_code=XTBOP0FA00Q02009&start_period=2005-01-01&end_period=2021-01-01&sort_by=asc", headers=headers)
### portfolio flow (net) USD: series_code=XTBOP0FA00Q02256&start_period=2005-01-01&end_period=2021-01-01&sort_by=asc", headers=headers)
### portfolio flow (net) THB: series_code=XTBOP0FA00Q02008&start_period=2005-01-01&end_period=2021-01-01&sort_by=asc", headers=headers)
### BOP USD: series_code=XTBOP0OVL0Q02371&start_period=2005-01-01&end_period=2021-01-01&sort_by=asc", headers=headers)
### BOP THB: series_code=XTBOP0OVL0Q02123&start_period=2005-01-01&end_period=2021-01-01&sort_by=asc", headers=headers)

### reserve_gross USD: series_code=XTRSV00000M00698&start_period=1993-01-01&end_period=2021-01-01&sort_by=asc", headers=headers)
### reserve_gross THB: series_code=XTRSV00000M00704&start_period=1993-01-01&end_period=2021-01-01&sort_by=asc", headers=headers)

### gov bond yield_2y: series_code=FMRTINTM00031&start_period=2005-01-01&end_period=2021-01-01&sort_by=asc", headers=headers)
### gov bond yield_5y: series_code=FMRTINTM00034&start_period=2005-01-01&end_period=2021-01-01&sort_by=asc", headers=headers)
### gov bond yield_10y: series_code=FMRTINTM00039&start_period=2005-01-01&end_period=2021-01-01&sort_by=asc", headers=headers)

### commercial bank SM: series_code=FINPQ00228&start_period=2007-01-01&end_period=2021-01-01&sort_by=asc", headers=headers)
### commercial bank NPL: series_code=FINPQ00229&start_period=2007-01-01&end_period=2021-01-01&sort_by=asc", headers=headers)

### condo price: series_code=EIRPPIQ00085&start_period=2008-01-01&end_period=2021-01-01&sort_by=asc", headers=headers)
### townhouse with land price: series_code=EIRPPIQ00083&start_period=2008-01-01&end_period=2021-01-01&sort_by=asc", headers=headers)
### land price: series_code=EIRPPIQ00087&start_period=2008-01-01&end_period=2021-01-01&sort_by=asc", headers=headers)
### house price: series_code=EIRPPIQ00081&start_period=2008-01-01&end_period=2021-01-01&sort_by=asc", headers=headers)

### new loan rate: series_code=FMRTINTM00172&start_period=2005-01-01&end_period=2021-01-01&sort_by=asc", headers=headers)

TimeoutError: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond

In [3]:
# Policy Rates

## Data source Overview
## BIS.cbpol : "XM"(EU), "GB","JP", "US", "SG", "MY", "PH", "ID", "KR", "HK","CN", "AU", "SG (before 2007-05)"
## Taiwan central bank (CBC): "TW"
## Singapore central bank (MAS): SG after 2007-05

### BIS central bank policy rate
countries_list = ["XM", "GB","JP", "US", "SG", "MY", "PH", "ID", "KR", "HK","CN", "AU"]
bis_df = fetch_series("BIS", "cbpol", max_nb_series=100 ,dimensions={
    "REF_AREA": countries_list,
    "FREQ": ["M"], # taiwan is missing
})
bis_df = bis_df[['REF_AREA','period','value',]]
bis_df = bis_df.pivot_table(index=["period"], columns = 'REF_AREA', values = 'value')
bis_df = bis_df.rename(columns={'XM': 'EU'}, index={'GB': 'UK'})

#### Taiwan
link ='https://www.cbc.gov.tw/public/data/Ebanking/EINTEREST-H.xls'
tw_df = pd.read_excel(link,'EINTEREST-H')
tw_df = tw_df[['Unnamed: 0','Unnamed: 1']]
tw_df = tw_df.loc[2:]
tw_df = tw_df.rename(columns={'Unnamed: 0': 'period','Unnamed: 1': 'TW'}).dropna()
tw_df['period'] = tw_df['period'].apply(lambda x:str(x)).apply(lambda x: datetime.strptime(x,'%Y.%m'))
tw_df  = tw_df.set_index('period')

#### Singapore
#generating whole series of data
sg_df = requests.get(url='https://eservices.mas.gov.sg/api/action/datastore/search.json?resource_id=9a0bf149-308c-4bd2-832d-76c8e6cb47ed&fields=end_of_day,sora&offset=6600&sort=end_of_day%20asc')
sg_df = sg_df.json()
sg_df = pd.DataFrame(sg_df['result']['records']).rename(columns={'sora': "SG", "end_of_day": "period"})
sg_df = sg_df[['period', 'SG']]

i=6600 #<< 2022 = offset value where 1993-m1 begin, 6600 = offset value where series starts
while True:
    try:
        i = i+100
        request_url = "https://eservices.mas.gov.sg/api/action/datastore/search.json?resource_id=9a0bf149-308c-4bd2-832d-76c8e6cb47ed&fields=end_of_day,sora&offset=" + str(i) + "&sort=end_of_day%20asc" 
        sg_df_temp = requests.get(url=request_url)
        sg_df_temp = sg_df_temp.json()
        sg_df_temp = pd.DataFrame(sg_df_temp['result']['records']).rename(columns={'sora': "SG", "end_of_day": "period"})
        sg_df_temp = sg_df_temp[['period', 'SG']]
        sg_df = sg_df.append(sg_df_temp)
    except KeyError:
        break
        
sg_df = sg_df.dropna()
sg_df['period'] = pd.to_datetime(sg_df['period'])
sg_df = sg_df.set_index('period')
sg_df_daily = sg_df
sg_df = sg_df_daily.resample('1M').mean()
sg_df = sg_df.reset_index()
sg_df['period'] = sg_df['period'].apply(lambda x: x.strftime('%Y-%m'))
sg_df['period'] = pd.to_datetime(sg_df['period'],format = '%Y.%m')
sg_df.set_index('period', inplace=True)


## Merging Policy data
pol_data = pd.merge(bis_df,sg_df, on=['period'], how = 'left')
pol_data = pd.merge(pol_data,tw_df, on=['period'], how = 'left')
pol_data = pol_data.loc['1993-01-01':]

In [4]:
# Export output to excel 

## create excel writer
writer = pd.ExcelWriter(time.strftime('%Y-%m')+'-API_output.xlsx')
# write dataframe to excel sheets
quarterly_data.to_excel(writer, 'quarterly_data')
monthly_data.to_excel(writer, 'monthly_data')
pol_data.to_excel(writer, 'pol_rate')
# save the excel file
writer.save()
print('DataFrame is written successfully to Excel Sheet.')

DataFrame is written successfully to Excel Sheet.
